In [1]:
import urllib.request
from bs4 import BeautifulSoup

In [2]:
articleUrl = 'https://globoesporte.globo.com/natacao/noticia/estudo-dieta-e-menos-treinos-os-segredos-de-nicholas-para-ser-o-melhor-do-mundo-aos-38-anos.ghtml'

In [3]:
page = urllib.request.urlopen(articleUrl).read().decode('utf8', 'ignore')
soup = BeautifulSoup(page, 'lxml')

In [4]:
text = ' '.join(map(lambda p: p.text, soup.find_all('article')))

In [6]:
text.encode('ascii', errors='replace')
text.replace('\n', ' ')

'               Nicholas Santos é ouro na prova dos 50m borboleta no Mundial de piscina curta de natação          Nicholas Santos gosta de provar que, aos 38 anos, a idade é apenas um número para se deixar de lado. Neste sábado, com a marca de 21s81, o paulista conquistou a medalha de ouro nos 50m borboleta no Campeonato Mundial em piscina curta (25m) de Hangzhou, na China. Por trás do sucesso, há muito pouco de acaso e muito de trabalho.      O nadador entendeu os limites do corpo e o que fazer para mantê-lo sadio e longe de lesões. O primeiro passo foi restringir excessos. Formado em fisioterapia, ele usou dos conhecimentos para traçar uma rotina espartana e fundamental para sua longevidade.     Nicholas não come doces, segue uma dieta macrobiótica rígida. Reduziu a carga de treinos consideravelmente e passou a se concentrar em uma prova: os 50m borboleta, que para seu infortúnio não faz parte do programa dos Jogos Olímpicos.                Nicholas Santos festeja a medalha de ouro: 

In [8]:
import urllib.request
from bs4 import BeautifulSoup

def get_text_web(url):
    page = urllib.request.urlopen(url).read().decode('utf8')
    soup = BeautifulSoup(page, 'lxml')
    text = ' '.join(map(lambda p: p_text, soup.find_all('article')))
    text.encode('ascii', errors='replace')
    text.replace('\n', ' ')
    return text

In [9]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from string import punctuation

In [10]:
sents = sent_tokenize(text)

In [11]:
words_sent = word_tokenize(text)

In [14]:
stopwords = set(stopwords.words('portuguese') + list(punctuation))
stopwords.add("é")

In [15]:
# Retirando stopwords
words_sent = [word for word in words_sent if word not in stopwords]

In [16]:
# Verificando a frequência de cada palavra
from nltk.probability import FreqDist

freq = FreqDist(words_sent)
freq

FreqDist({'Nicholas': 6, '50m': 5, 'O': 5, 'Santos': 4, 'prova': 4, 'borboleta': 4, 'Mundial': 4, 'piscina': 4, 'corpo': 4, 'ouro': 3, ...})

In [17]:
# Encontrar palavras com maior frequência
from heapq import nlargest

In [18]:
nlargest(10, freq, key=freq.get)

['Nicholas',
 '50m',
 'O',
 'Santos',
 'prova',
 'borboleta',
 'Mundial',
 'piscina',
 'corpo',
 'ouro']

In [19]:
from collections import defaultdict

ranking = defaultdict(int)

for i, sent in enumerate(sents):
    for w in word_tokenize(sent.lower()):
        if w in freq:
            ranking[i] += freq[w]
ranking

defaultdict(int,
            {0: 41,
             1: 34,
             2: 5,
             3: 13,
             4: 4,
             5: 10,
             6: 6,
             7: 23,
             8: 22,
             9: 10,
             10: 7,
             11: 33,
             12: 12,
             13: 18,
             14: 13,
             15: 9,
             16: 37,
             17: 23,
             18: 13,
             19: 10,
             20: 14,
             21: 14,
             22: 2,
             23: 3,
             24: 6,
             25: 11,
             26: 17,
             27: 3,
             28: 21,
             29: 9})

In [23]:
# Extraindo índice das 4 palavras que possuem maior frequência no artigo
sents_idx = nlargest(4, ranking, key=ranking.get)
sents_idx

[0, 16, 1, 11]

In [24]:
# Indexando as frases (sentenças) através do índice das palavras que mais aparecem no artigo
[sents[j] for j in sorted(sents_idx)]

['               Nicholas Santos é ouro na prova dos 50m borboleta no Mundial de piscina curta de natação          Nicholas Santos gosta de provar que, aos 38 anos, a idade é apenas um número para se deixar de lado.',
 'Neste sábado, com a marca de 21s81, o paulista conquistou a medalha de ouro nos 50m borboleta no Campeonato Mundial em piscina curta (25m) de Hangzhou, na China.',
 '- Por eu ter me formado em Fisioterapia, estar em uma empresa de treinamento, em contato com os maiores profissionais na área de treinamento e educação física, hoje entendo o que meu corpo precisa para nadar bem - afirmou ao GloboEsporte.com após o título.',
 'Assim ele construiu esse nadador incansável que, neste sábado, caiu na água duas vezes em 40 minutos para conquistar dois pódios - além dos 50m borboleta, também ajudou o revezamento 4x50m medley masculino a levar o bronze.']

In [28]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from nltk.probability import FreqDist
from heapq import nlargest
from collections import defaultdict

def summarize(text, n):
    sents = sent_tokenize(text)
    
    assert n <= len(sents)
    word_sent = word_tokenize(text.lower())
    _stopwords = set(stopwords.words('portuguese') + list(punctuation))
    _stopwords.add('é')
    _stopwords.add('á')
    _stopwords.add('ã')
    _stopwords.add('ê')
    
    word_sent = [word for word in word_sent if word not in _stopwords]
    freq = FreqDist(word_sent)
    
    ranking = defaultdict(int)
    
    for i, sent in enumerate(sents):
        for w in word_tokenize(sent.lower()):
            if w in freq:
                ranking[i] = freq[w]
    
    sents_idx = nlargest(n, ranking, key=ranking.get)
    return [sents[j] for j in sorted(sents_idx)]

In [30]:
summarize(text, 4)

['Neste sábado, com a marca de 21s81, o paulista conquistou a medalha de ouro nos 50m borboleta no Campeonato Mundial em piscina curta (25m) de Hangzhou, na China.',
 'Além disso, tem de encaixar na agenda os momentos com o filho, também chamado Nicholas.',
 'O maior desafio era nadar o revezamento, recuperar e vir para a prova.',
 'Quer curtir o dia, as medalhas e saborear a consciência de ainda estar entre os melhores do mundo, mesmo contra nadadores com metade de sua idade.']